# IIRC Package Tutorial Using PyTorch

In [1]:
import numpy as np
from PIL import Image

from iirc.lll_dataset.torch_lll_dataset import TorchLLLDataset
from iirc.iirc_definitions import IIRC_SETUP, CIL_SETUP, NO_LABEL_PLACEHOLDER

## Using CIL setup (Class Incremental Learning)

Let's create a mock dataset of 120 samples, with each belonging to one of the four classes **A** - **B** - **C** - **D**

In [2]:
n = 120
n_per_y = 30
x = np.random.rand(n,32,32,3)
y = ["A"] * n_per_y + ["B"] * n_per_y + ["C"] * n_per_y + ["D"] * n_per_y

Now this dataset should be converted to the format used by the lll_datasets, where images should be a pillow images (or strings representing the images path in case of large datasets, such as ImageNet), and the dataset should be arranged as a list of tuples of the form (image, (label,))

In [3]:
x = np.uint8(x*255)
mock_dataset = [(Image.fromarray(x[i], 'RGB'), (y[i],)) for i in range(n)]

Now let's create a tasks schedule, where the first task introduces the labels **A** and **B**, and the second task introduces **C** and **D** (tasks don't need to be of equal size)

In [4]:
tasks = [["A", "B"], ["C", "D"]]

We also need a transformations function that takes the image and converts it to a tensor, as well as normalize the image, apply augmentations, etc.

There are two such functions that can be provided: *essential_transforms_fn* and *augmentation_transforms_fn*

If *augmentation_transforms_fn* is provided, it will always be applied except if the *TorchLLLDataset* is told not to apply augmentations in a specific context (we will see later how)

Otherwise, *essential_transforms_fn* will be applied

So for example in a test set where augmentations are not needed, *augmentation_transforms_fn* shouldn't be provided in the *TorchLLLDataset* initialization

In [5]:
import torchvision.transforms as transforms

essential_transforms_fn = transforms.ToTensor()
augmentation_transforms_fn = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

Now we are ready to initialize the incremental dataset 

In [6]:
cil_dataset = TorchLLLDataset(dataset=mock_dataset, tasks=tasks, setup=CIL_SETUP, 
                              essential_transforms_fn=essential_transforms_fn, 
                              augmentation_transforms_fn=augmentation_transforms_fn)

Then we can make use of *cil_dataset* by choosing the task we wish to train on using the *choose_task(task_id)* function, and creating a dataloader out of it

In [7]:
cil_dataset.choose_task(0) 

If we print the length of the dataset, we will only get the length of the samples that belong to the current task. The same goes for fetching a sample, as we only have access to the samples of the current task. This means that indexing the *cil_dataset* is relative to the current task, so the 0th sample for example will be different when we choose another task

In [8]:
print(len(cil_dataset))

60


In [9]:
cil_dataset[0]

(tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.5137, 0.7647, 0.7412],
          [0.0000, 0.0000, 0.0000,  ..., 0.1725, 0.4000, 0.9922],
          [0.0000, 0.0000, 0.0000,  ..., 0.2549, 0.7137, 0.3373]],
 
         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.7804, 0.6314, 0.9529],
          [0.0000, 0.0000, 0.0000,  ..., 0.8471, 0.8471, 0.8196],
          [0.0000, 0.0000, 0.0000,  ..., 0.1098, 0.6000, 0.4235]],
 
         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ...,

As we have seen the format returned when we fetch a sample from the dataset is a tuple of length 3 consisting of (image tensor, image label, NO_LABEL_PLACEHOLDER), NO_LABEL_PLACEHOLDER is set to the string "None" and should be ignored in the class incremental setup.

We can also access what classes have we seen thus far (they don't reset if a previous task is re-chosen, also they are unordered):

In [10]:
cil_dataset.reset()
cil_dataset.choose_task(0) 
print(f"Task {cil_dataset.cur_task_id}, current task: {cil_dataset.cur_task}, dataset length: {len(cil_dataset)}, classes seen: {cil_dataset.seen_classes}")
cil_dataset.choose_task(1)
print(f"Task {cil_dataset.cur_task_id}, current task: {cil_dataset.cur_task}, dataset length: {len(cil_dataset)}, classes seen: {cil_dataset.seen_classes}")

Task 0, current task: ['A', 'B'], dataset length: 60, classes seen: ['B', 'A']
Task 1, current task: ['C', 'D'], dataset length: 60, classes seen: ['D', 'B', 'C', 'A']


If we need to load the data of all the tasks up to a specific task (including it), this can be done using the *load_tasks_up_to(task_id)* function

In [11]:
cil_dataset.load_tasks_up_to(1)
print(f"current task: {cil_dataset.cur_task}, dataset length: {len(cil_dataset)}")

current task: ['A', 'B', 'C', 'D'], dataset length: 120


If a sample needs to be accessed in it's original form (PIL form) without any transformations, for example to be added to the replay buffer, the *get_item(index)* method can be used:

In [12]:
cil_dataset.get_item(0)

(<PIL.Image.Image image mode=RGB size=32x32 at 0x286B762A460>, 'A', 'None')

and if we need to know the indices of the samples that belong to a specific class, this can be done using the *get_image_indices_by_cla(class name)* method, however, this can only be done if that class belongs to the current task. Moreover, these indices are relative to the current task, so whenever we change the task, they would point to totally different samples in the new task

In [13]:
cil_dataset.get_image_indices_by_cla("A")

array([17,  7, 29, 26, 13,  2,  6, 15, 27, 21, 19, 10,  5,  9,  4, 14,  3,
       22, 20,  0, 23, 28, 18, 11, 12,  8, 16, 24, 25,  1])

If *cil_dataset* uses data augmentations, but we needed to disable them in a specific part of the code, this context manager can be used:

In [14]:
with cil_dataset.disable_augmentations():
    # any samples loaded here will have the essential transformations function applied to them
    pass

Finally, to load this dataset in minibatches for training, the torch dataloader can be used with it, **but don't forget to reinstantiate the dataloader whenever the task changes**

In [15]:
from torch.utils.data import DataLoader
cil_dataset.choose_task(0) 
train_loader = DataLoader(cil_dataset, batch_size=2, shuffle=True)
next(iter(train_loader))

[tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.6235, 0.8745, 0.3255],
           [0.0000, 0.0000, 0.0000,  ..., 0.2118, 0.9804, 0.7490],
           [0.0000, 0.0000, 0.0000,  ..., 0.0902, 0.4235, 0.9373]],
 
          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.1882, 0.4745, 0.4510],
           [0.0000, 0.0000, 0.0000,  ..., 0.4353, 0.6039, 0.6863],
           [0.0000, 0.0000, 0.0000,  ..., 0.2118, 0.5373, 0.1333]],
 
          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.00

## Using IIRC setup (Incremental Implicitly Refined Classification)

Now Imaging that samples belonging to class **A** have also another sublabel of either **Aa** or **Ab**, and the samples belonging to class **B** have another sublabel of either **Ba** or **Bb**

In [16]:
y_iirc = []
for i, label in enumerate(y):
    if label == "A" and (i % 2) == 0:
        y_iirc.append(("A", "Aa"))
    elif label == "A":
        y_iirc.append(("A", "Ab"))
    elif label == "B" and (i % 2) == 0:
        y_iirc.append(("B", "Ba"))
    elif label == "B":
        y_iirc.append(("B", "Bb"))
    else:
        y_iirc.append((label,))
print(y_iirc)

[('A', 'Aa'), ('A', 'Ab'), ('A', 'Aa'), ('A', 'Ab'), ('A', 'Aa'), ('A', 'Ab'), ('A', 'Aa'), ('A', 'Ab'), ('A', 'Aa'), ('A', 'Ab'), ('A', 'Aa'), ('A', 'Ab'), ('A', 'Aa'), ('A', 'Ab'), ('A', 'Aa'), ('A', 'Ab'), ('A', 'Aa'), ('A', 'Ab'), ('A', 'Aa'), ('A', 'Ab'), ('A', 'Aa'), ('A', 'Ab'), ('A', 'Aa'), ('A', 'Ab'), ('A', 'Aa'), ('A', 'Ab'), ('A', 'Aa'), ('A', 'Ab'), ('A', 'Aa'), ('A', 'Ab'), ('B', 'Ba'), ('B', 'Bb'), ('B', 'Ba'), ('B', 'Bb'), ('B', 'Ba'), ('B', 'Bb'), ('B', 'Ba'), ('B', 'Bb'), ('B', 'Ba'), ('B', 'Bb'), ('B', 'Ba'), ('B', 'Bb'), ('B', 'Ba'), ('B', 'Bb'), ('B', 'Ba'), ('B', 'Bb'), ('B', 'Ba'), ('B', 'Bb'), ('B', 'Ba'), ('B', 'Bb'), ('B', 'Ba'), ('B', 'Bb'), ('B', 'Ba'), ('B', 'Bb'), ('B', 'Ba'), ('B', 'Bb'), ('B', 'Ba'), ('B', 'Bb'), ('B', 'Ba'), ('B', 'Bb'), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C',), ('C

In [17]:
mock_dataset_iirc = [(Image.fromarray(x[i], 'RGB'), y_iirc[i]) for i in range(n)]

In [18]:
mock_dataset_iirc[0]

(<PIL.Image.Image image mode=RGB size=32x32 at 0x286B76358E0>, ('A', 'Aa'))

let's redefine the tasks by incorporating the new subclasses

In [19]:
tasks_iirc = [["A", "B", "C"], ["Aa", "Ba", "D"], ["Ab", "Bb"]]

All the functionality that we mentioned in the CIL setup is still applicable here, but there are some differences though.

The first difference is that in the training set, we don't necessarily need all the samples that belong to both labels **A** and **Aa** to be seen across the two tasks, what makes more sense is that some of them need to appear in the first task and some others need to appear in the second task, probably with some overlap

Hence we have the two arguments *superclass_data_pct* and *subclass_data_pct*:
* *superclass_data_pct* controls the percentage of the samples that belong to **A** that will appear when **A** is introduced
* *subclass_data_pct* controls the percentage of the samples that belong to **Aa** that will appear when **Aa** is introduced (same for other subclasses)

So in this example we have 15 samples that have the labels (**A**, **Aa**), and 15 samples that have the labels (**A**, **Ab**):
* *superclass_data_pct* = 1.0, *subclass_data_pct* = 1.0: 
This means that all 30 samples with label **A** will appear in the first task, then all 15 samples with label **Aa** will appear in the second task, etc (100% overlab)
* *superclass_data_pct* = 0.5, *subclass_data_pct* = 0.5: 
15 samples with label **A** will appear in the first task, then 8 samples with label **Aa** will appear in the second task, etc (no overlab)
* *superclass_data_pct* = 0.6, *subclass_data_pct* = 0.8: 
18 samples with label **A** will appear in the first task, then 12 samples with label **Aa** will appear in the second task, etc (40% overlab)

This procedure will only be done if the argument *test_mode* is set to *False*, as in the test set we don't care about this kind of data repetition but we care more about evaluating on all the available samples

In [20]:
iirc_dataset_train = TorchLLLDataset(dataset=mock_dataset_iirc, tasks=tasks_iirc, setup=IIRC_SETUP, test_mode=False,
                                     essential_transforms_fn=essential_transforms_fn, 
                                     augmentation_transforms_fn=augmentation_transforms_fn, 
                                     superclass_data_pct=0.6, subclass_data_pct=0.8)

iirc_dataset_test = TorchLLLDataset(dataset=mock_dataset_iirc, tasks=tasks_iirc, setup=IIRC_SETUP, test_mode=True,
                                     essential_transforms_fn=essential_transforms_fn, 
                                     augmentation_transforms_fn=augmentation_transforms_fn)

iirc_dataset_train.choose_task(0)
iirc_dataset_test.choose_task(0)

print(f"Length of task 0 training data: {len(iirc_dataset_train)}, number of samples for class A: {len(iirc_dataset_train.get_image_indices_by_cla('A'))}")
print(f"Length of task 0 training data: {len(iirc_dataset_train)}, number of samples for class A: {len(iirc_dataset_test.get_image_indices_by_cla('A'))}")


Length of task 0 training data: 66, number of samples for class A: 18
Length of task 0 training data: 66, number of samples for class A: 30


The second difference is the concept of incomplete information, so typically in the IIRC setup, the model is not told all the labels of a sample, but only the labels that correspond to the current task, and the model should figure the other labels on its own.

This only applies to the training set though, as for the test set you need to know all the labels to evaluate the model properly

In [21]:
labels_ = ["A", "Aa", "Ab"]
iirc_dataset_train.reset()
iirc_dataset_test.reset()
for task_id, label in enumerate(labels_):
    iirc_dataset_train.choose_task(task_id)
    iirc_dataset_test.choose_task(task_id)
    train_sample, train_label1, train_label2 = \
        iirc_dataset_train[iirc_dataset_train.get_image_indices_by_cla(label, num_samples=1)[0]]
    test_sample, test_label1, test_label2 = \
        iirc_dataset_test[iirc_dataset_test.get_image_indices_by_cla(label, num_samples=1)[0]]
    
    print(f"task {task_id}:\ntraining sample label: {(train_label1, train_label2)}")
    print(f"test sample label: {test_label1, test_label2}\n")

task 0:
training sample label: ('A', 'None')
test sample label: ('A', 'None')

task 1:
training sample label: ('Aa', 'None')
test sample label: ('Aa', 'A')

task 2:
training sample label: ('Ab', 'None')
test sample label: ('A', 'Ab')



To make a dataset use the *complete information* mode irrespective of whether it is a training set or a test set, the argument *complete_information_mode* can be provided when initializing the dataset, and the functions *enable_complete_information_mode()* and *enable_incomplete_information_mode()* can be used as well after initialization.

Take note that the *load_tasks_up_to(task_id)* function doesn't work in the *incomplete information*

## Usage with Incremental-CIFAR100, IIRC-CIFAR100, Incremental-Imagenet, IIRC-Imagenet

In [22]:
from iirc.lll_datasets_loader import get_lll_datasets
from iirc.iirc_definitions import PYTORCH
from iirc.utils.download_cifar import download_extract_cifar100

For using these datasets with the preset tasks schedules, the original *CIFAR100* and/or *ImageNet2012* need to be downloaded first.

In the case of *CIFAR100*, the dataset can be downloaded using the following method

In [25]:
download_extract_cifar100("./data")

extracting CIFAR 100
dataset extracted


E:\Work\Master_Research\ContinualRefinement\ContinualRefinement\iirc\utils\download_cifar.py:15: UserWarning: ./data\cifar-100-python.tar.gz already exists, the dataset won't be download
  warnings.warn(f"{target_path} already exists, the dataset won't be download")


In the case of *ImageNet*, it has to be downloaded manually, and be arranged in the following manner:
* Imagenet
    * train
        * n01440764
        * n01443537
        * ...
    * val
        * n01440764
        * n01443537
        * ...
    

Then the *get_lll_datasets* function should be used. The tasks schedules/configurations preset per dataset are:

* *Incremental-CIFAR100*: 10 configurations, each starting with 50 classes in the first task, followed by 10 tasks each having 5 classes
* *IIRC-CIFAR100*: 10 configurations, each starting with 10 superclasses in the first task, followed by 21 tasks each having 5 classes
* *Incremental-Imagenet-full*: 5 configurations, each starting with 160 classes in the first task, followed by 28 tasks each having 30 classes
* *Incremental-Imagenet-lite*: 5 configurations, each starting with 160 classes in the first task, followed by 9 tasks each having 30 classes
* *IIRC-Imagenet-full*: 5 configurations, each starting with 63 superclasses in the first task, followed by 34 tasks each having 30 classes
* *IIRC-Imagenet-lite*: 5 configurations, each starting with 63 superclasses in the first task, followed by 9 tasks each having 30 classes

Although these configurations might seem they are limiting the choices, but the point here is to have a standard set of tasks and class orders so that the results are comparable across different works, otherwise if needed, new task configurations can be added manually as well in the *metadata* folder

In [30]:
# The datasets supported are ("incremental_cifar100", "iirc_cifar100", "incremental_imagenet_full", "incremental_imagenet_lite", 
# "iirc_imagenet_full", "iirc_imagenet_lite")
lll_datasets, tasks, class_names_to_idx = get_lll_datasets(dataset_name = "iirc_cifar100",
                                                           dataset_root = "./data", # the parent directory of cifar-100-python or the imagenet folders
                                                           setup = IIRC_SETUP,
                                                           framework = PYTORCH,
                                                           tasks_configuration_id = 0,
                                                           essential_transforms_fn = essential_transforms_fn,
                                                           augmentation_transforms_fn = augmentation_transforms_fn,
                                                           joint = False 
                                                          )

Creating iirc_cifar100
Setup used: IIRC
Using PyTorch
Dataset created


*joint* can also be set to *True* in case of joint training (all classes will come in one task)

The result of the previous function has the following form:

In [31]:
lll_datasets # four splits

{'train': <iirc.lll_dataset.torch_lll_dataset.TorchLLLDataset at 0x286ba877070>,
 'intask_valid': <iirc.lll_dataset.torch_lll_dataset.TorchLLLDataset at 0x286b8835fd0>,
 'posttask_valid': <iirc.lll_dataset.torch_lll_dataset.TorchLLLDataset at 0x286b8835f70>,
 'test': <iirc.lll_dataset.torch_lll_dataset.TorchLLLDataset at 0x286b8835dc0>}

In [35]:
print(tasks[:3])

[['flowers', 'small_mammals', 'trees', 'aquatic_mammals', 'fruit_and_vegetables', 'people', 'food_containers', 'vehicles', 'large_carnivores', 'insects'], ['television', 'spider', 'shrew', 'mountain', 'hamster'], ['road', 'poppy', 'household_furniture', 'woman', 'bee']]


In [37]:
print(class_names_to_idx)

{'flowers': 0, 'small_mammals': 1, 'trees': 2, 'aquatic_mammals': 3, 'fruit_and_vegetables': 4, 'people': 5, 'food_containers': 6, 'vehicles': 7, 'large_carnivores': 8, 'insects': 9, 'television': 10, 'spider': 11, 'shrew': 12, 'mountain': 13, 'hamster': 14, 'road': 15, 'poppy': 16, 'household_furniture': 17, 'woman': 18, 'bee': 19, 'tulip': 20, 'clock': 21, 'orange': 22, 'beaver': 23, 'rocket': 24, 'bicycle': 25, 'can': 26, 'squirrel': 27, 'wardrobe': 28, 'bus': 29, 'whale': 30, 'sweet_pepper': 31, 'telephone': 32, 'leopard': 33, 'bowl': 34, 'skyscraper': 35, 'baby': 36, 'cockroach': 37, 'boy': 38, 'lobster': 39, 'motorcycle': 40, 'forest': 41, 'tank': 42, 'orchid': 43, 'chair': 44, 'crab': 45, 'girl': 46, 'keyboard': 47, 'otter': 48, 'bed': 49, 'butterfly': 50, 'lawn_mower': 51, 'snail': 52, 'caterpillar': 53, 'wolf': 54, 'pear': 55, 'tiger': 56, 'pickup_truck': 57, 'cup': 58, 'reptiles': 59, 'train': 60, 'sunflower': 61, 'beetle': 62, 'apple': 63, 'palm_tree': 64, 'plain': 65, 'larg

*lll_datasets* has four splits, where *train* is for training, *intask_valid* is for validation during task training (in case of IIRC setup, this split is using *incomplete information* like the *train* split), *posttask_valid* is for validation after each task training (in case of IIRC setup, this split is using *complete information* like the *test* split), and finally the *test* split